In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='c617d2f3-d61b-48fa-bfbf-092035fd8e8a', project_access_token='p-1e0099079288ca40cc7f2e215a32d023063be00f')
pc = project.project_context

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_60016c44ce56487ea8245c5805812373 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='9shLa8C9HC2HIq4GnbVe4_6rZpbQF33ImqeyOxpe6JHE',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_60016c44ce56487ea8245c5805812373.get_object(Bucket='courseracapstone-donotdelete-pr-xzmqu7pelpsf9d',Key='csvjson.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Please read the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' to learn more about the possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html




In [4]:
gb = pd.read_json(body, orient='values')
gb

,AREA CODE(S),COUNTY,POPULATION,TYPE,ZIP CODE
0,920,Brown,4095,Standard,54173
1,920,Brown,4306,Standard,54229
2,920 / 715,Brown,22742,Standard,54301
3,920 / 715,Brown,30611,Standard,54302
4,920 / 715,Brown,27041,Standard,54303
5,920,Brown,28153,Standard,54304
6,920 / 715,Brown,0,PO Box,54305
7,920 / 715,Brown,0,PO Box,54306
8,920 / 715,Brown,1092,PO Box,54307
9,920 / 715,Brown,0,PO Box,54308


In [5]:
list(gb)

['AREA CODE(S)', 'COUNTY', 'POPULATION', 'TYPE', 'ZIP CODE']

In [6]:
print('The dataframe has {} zip codes.'.format(
        len(gb['ZIP CODE'].unique()),
        gb.shape[0]
    )
)

The dataframe has 14 zip codes.


## Clean Data

In [7]:
gbzip = pd.DataFrame(gb, columns = ['POPULATION' , 'TYPE', 'ZIP CODE'])
# remove columns containing area code and county data since they are not useful
# keep population to determine relative restaurant traffic by immediate surroundings
gbzip

,POPULATION,TYPE,ZIP CODE
0,4095,Standard,54173
1,4306,Standard,54229
2,22742,Standard,54301
3,30611,Standard,54302
4,27041,Standard,54303
5,28153,Standard,54304
6,0,PO Box,54305
7,0,PO Box,54306
8,1092,PO Box,54307
9,0,PO Box,54308


In [8]:
columns = ['POPULATION', 'TYPE', 'ZIPCODE']
gb_data = pd.DataFrame(columns = columns, index = [0,1,2,3,4,5,6,7])
gb_data

,POPULATION,TYPE,ZIPCODE
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN


In [9]:
gb_data = gb_data.append({'POPULATION' : 5451 , 'TYPE' : 'Standard', 'ZIPCODE' : 54155} , ignore_index=True)
gb_data = gb_data.append({'POPULATION' : 4306 , 'TYPE' : 'Standard', 'ZIPCODE' : 54229} , ignore_index=True)
gb_data = gb_data.append({'POPULATION' : 22742 , 'TYPE' : 'Standard', 'ZIPCODE' : 54301} , ignore_index=True)
gb_data = gb_data.append({'POPULATION' : 30611 , 'TYPE' : 'Standard', 'ZIPCODE' : 54302} , ignore_index=True)
gb_data = gb_data.append({'POPULATION' : 27041 , 'TYPE' : 'Standard', 'ZIPCODE' : 54303} , ignore_index=True)
gb_data = gb_data.append({'POPULATION' : 28153 , 'TYPE' : 'Standard', 'ZIPCODE' : 54304} , ignore_index=True)
gb_data = gb_data.append({'POPULATION' : 33580 , 'TYPE' : 'Standard', 'ZIPCODE' : 54311} , ignore_index=True)
gb_data = gb_data.append({'POPULATION' : 35897 , 'TYPE' : 'Standard', 'ZIPCODE' : 54313} , ignore_index=True)

gb_data = gb_data.drop(gb_data.index[[0,1,2,3,4,5,6,7]])

gb_data

,POPULATION,TYPE,ZIPCODE
8,5451,Standard,54155
9,4306,Standard,54229
10,22742,Standard,54301
11,30611,Standard,54302
12,27041,Standard,54303
13,28153,Standard,54304
14,33580,Standard,54311
15,35897,Standard,54313


## Split into East and West Subsets

In [10]:
east = gb_data['ZIPCODE']
west = gb_data['ZIPCODE']

In [11]:
east = east.drop(gb_data.index[[1,2,3,6]])
west = west.drop(gb_data.index[[0,4,5,7]])

print(east)
print(west)

8     54155
12    54303
13    54304
15    54313
Name: ZIPCODE, dtype: object
9     54229
10    54301
11    54302
14    54311
Name: ZIPCODE, dtype: object


In [12]:
columns = ['POPULATION', 'ZIPCODE', 'LATITUDE', 'LONGITUDE']
east_data = pd.DataFrame(columns = columns, index = [0,1,2,3])
east_data

,POPULATION,ZIPCODE,LATITUDE,LONGITUDE
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN


In [13]:
east_data = east_data.append({'POPULATION' : 5451 , 'ZIPCODE' : 54155, 'LATITUDE' : 44.5176 , 'LONGITUDE' : -88.2266 } , ignore_index=True)
east_data = east_data.append({'POPULATION' : 27041 , 'ZIPCODE' : 54303, 'LATITUDE' : 44.5432 , 'LONGITUDE' : -88.0608 } , ignore_index=True)
east_data = east_data.append({'POPULATION' : 28153 , 'ZIPCODE' : 54304, 'LATITUDE' : 44.4867 , 'LONGITUDE' : -88.0608 } , ignore_index=True)
east_data = east_data.append({'POPULATION' : 35897 , 'ZIPCODE' : 54313, 'LATITUDE' : 44.5911 , 'LONGITUDE' : -88.1096 } , ignore_index=True)

east_data = east_data.drop(east_data.index[[0,1,2,3]])

east_data

,POPULATION,ZIPCODE,LATITUDE,LONGITUDE
4,5451.0,54155.0,44.5176,-88.2266
5,27041.0,54303.0,44.5432,-88.0608
6,28153.0,54304.0,44.4867,-88.0608
7,35897.0,54313.0,44.5911,-88.1096


In [15]:
columns = ['POPULATION', 'ZIPCODE', 'LATITUDE', 'LONGITUDE']
west_data = pd.DataFrame(columns = columns, index = [0,1,2,3])
west_data

,POPULATION,ZIPCODE,LATITUDE,LONGITUDE
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN


In [16]:
west_data = west_data.append({'POPULATION' : 4306 , 'ZIPCODE' : 54229, 'LATITUDE' : 44.5486 , 'LONGITUDE' : -87.8353 } , ignore_index=True)
west_data = west_data.append({'POPULATION' : 22742 , 'ZIPCODE' : 54301, 'LATITUDE' : 44.4812 , 'LONGITUDE' : -88.0167 } , ignore_index=True)
west_data = west_data.append({'POPULATION' : 30611 , 'ZIPCODE' : 54302, 'LATITUDE' : 44.5088 , 'LONGITUDE' : -87.9776 } , ignore_index=True)
west_data = west_data.append({'POPULATION' : 33580 , 'ZIPCODE' : 54311, 'LATITUDE' : 44.4739 , 'LONGITUDE' : -87.8746 } , ignore_index=True)

west_data = west_data.drop(west_data.index[[0,1,2,3]])

west_data

,POPULATION,ZIPCODE,LATITUDE,LONGITUDE
4,4306.0,54229.0,44.5486,-87.8353
5,22742.0,54301.0,44.4812,-88.0167
6,30611.0,54302.0,44.5088,-87.9776
7,33580.0,54311.0,44.4739,-87.8746


## Initial Mapping of Green Bay

In [17]:
latitude = 44.5133
longitude = -88.0133

In [19]:
# create map of East Green Bay using latitude and longitude values
map_east = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(east_data['LATITUDE'], east_data['LONGITUDE'], east_data['ZIPCODE']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_east)  
    
map_east

In [20]:
# create map of West Green Bay using latitude and longitude values
map_west = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(west_data['LATITUDE'], west_data['LONGITUDE'], west_data['ZIPCODE']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_west)  
    
map_west

## Foursquare Exploration - East Side

In [21]:
CLIENT_ID = '4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3'
CLIENT_SECRET = '2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3
CLIENT_SECRET:2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF


In [24]:
# explore first zip code on the east side
east_data.loc[4, 'ZIPCODE']

54155.0

In [26]:
east1_latitude = east_data.loc[4, 'LATITUDE']
east1_longitude = east_data.loc[4, 'LONGITUDE']

In [73]:
LIMIT = 100
radius = 10000

url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    east1_latitude, 
    east1_longitude, 
    radius, 
    LIMIT)
url1


'https://api.foursquare.com/v2/venues/explore?&client_id=4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3&client_secret=2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF&v=20180605&ll=44.5176,-88.2266&radius=10000&limit=100'

In [93]:
results = requests.get(url1).json()
results

{'meta': {'code': 200, 'requestId': '5c64f1671ed2192b59264898'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb00854f964a520e5323ce3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/golfcourse_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e6941735',
         'name': 'Golf Course',
         'pluralName': 'Golf Courses',
         'primary': True,
         'shortName': 'Golf Course'}],
       'id': '4bb00854f964a520e5323ce3',
       'location': {'address': '897 Riverdale Dr',
        'cc': 'US',
        'city': 'Oneida',
        'country': 'United States',
        'distance': 6173,
        'formattedAddress': ['897 Riverdale Dr',
         'Oneida, WI 54155',
         'United States'],
        'labeledLatLngs': [{'label': 'display',
     

In [77]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brown County Golf Course,Golf Course,44.525430,-88.149597
1,Maplewood Meats,Butcher,44.573925,-88.163058
2,Thornberry Creek at Oneida,Golf Course,44.552320,-88.147691
3,Outagamie County Fairgrounds,Racetrack,44.518424,-88.331591
4,Kropp's Supper Club,American Restaurant,44.587981,-88.159287


In [80]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
Airport,1,1,1
Airport Service,1,1,1
American Restaurant,3,3,3
Bank,1,1,1
Bar,2,2,2
Burger Joint,2,2,2
Butcher,1,1,1
Casino,2,2,2
Coffee Shop,1,1,1


In [76]:
#none fall under the Italian category, move onto next zipcode

In [39]:
# explore second zip code on the east side
east_data.loc[5, 'ZIPCODE']

54303.0

In [40]:
east2_latitude = east_data.loc[5, 'LATITUDE']
east2_longitude = east_data.loc[5, 'LONGITUDE']

In [94]:
LIMIT = 100
radius = 10000

url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    east2_latitude, 
    east2_longitude, 
    radius, 
    LIMIT)
url2

'https://api.foursquare.com/v2/venues/explore?&client_id=4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3&client_secret=2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF&v=20180605&ll=44.5432,-88.0608&radius=10000&limit=100'

In [95]:
results = requests.get(url2).json()
results

{'meta': {'code': 200, 'requestId': '5c64f273db04f5453397c497'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b5c960ef964a5202d3829e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d118951735',
         'name': 'Grocery Store',
         'pluralName': 'Grocery Stores',
         'primary': True,
         'shortName': 'Grocery Store'}],
       'id': '4b5c960ef964a5202d3829e3',
       'location': {'address': '2400 Duck Creek Parkway',
        'cc': 'US',
        'city': 'Green Bay',
        'country': 'United States',
        'distance': 1322,
        'formattedAddress': ['2400 Duck Creek Parkway',
         'Green Bay, WI 54303',
         'United States'],
        'labeledLatLngs': [{'label

In [96]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Woodman's Food Market,Grocery Store,44.543132,-88.077466
1,Bay View family restaurant,Breakfast Spot,44.547576,-88.048477
2,Westside YMCA,Gym / Fitness Center,44.555886,-88.085911
3,Sunny's Chinese Noodle House,Chinese Restaurant,44.561530,-88.073504
4,Mills Fleet Farm,Department Store,44.536196,-88.073733


In [97]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
American Restaurant,5,5,5
Bar,3,3,3
Basketball Stadium,1,1,1
Beer Garden,1,1,1
Bookstore,1,1,1
Breakfast Spot,2,2,2
Brewery,5,5,5
Burger Joint,4,4,4
Café,2,2,2


In [56]:
#none fall under the italian restaurant category, move onto next zipcode

In [57]:
# explore third zip code on the east side
east_data.loc[6, 'ZIPCODE']

54304.0

In [58]:
east3_latitude = east_data.loc[6, 'LATITUDE']
east3_longitude = east_data.loc[6, 'LONGITUDE']

In [98]:
LIMIT = 100
radius = 10000

url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    east3_latitude, 
    east3_longitude, 
    radius, 
    LIMIT)
url3

'https://api.foursquare.com/v2/venues/explore?&client_id=4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3&client_secret=2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF&v=20180605&ll=44.4867,-88.0608&radius=10000&limit=100'

In [99]:
results = requests.get(url3).json()
results

{'meta': {'code': 200, 'requestId': '5c64f598dd57977f6381fe90'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b43afd8f964a52063e625e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/candystore_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d117951735',
         'name': 'Candy Store',
         'pluralName': 'Candy Stores',
         'primary': True,
         'shortName': 'Candy Store'}],
       'id': '4b43afd8f964a52063e625e3',
       'location': {'address': '784 Willard Dr',
        'cc': 'US',
        'city': 'Green Bay',
        'country': 'United States',
        'crossStreet': 'Oneida',
        'distance': 172,
        'formattedAddress': ['784 Willard Dr (Oneida)',
         'Green Bay, WI 54304',
         'United States'],
        'labeledLat

In [100]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Seroogy's Chocolates,Candy Store,44.485840,-88.062602
1,Victoria's Secret PINK,Lingerie Store,44.484065,-88.065087
2,Marcus Bay Park Cinema,Movie Theater,44.484569,-88.062346
3,Gloria Jean's Coffees,Coffee Shop,44.484551,-88.066108
4,Lambeau Field,Football Stadium,44.501328,-88.062176


In [87]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
Airport,1,1,1
Airport Service,1,1,1
American Restaurant,3,3,3
Bank,1,1,1
Bar,2,2,2
Burger Joint,2,2,2
Butcher,1,1,1
Casino,2,2,2
Coffee Shop,1,1,1


In [64]:
#none fall under the italian restaurant category, move onto next zipcode

In [65]:
# explore fourth and last zip code on the east side
east_data.loc[7, 'ZIPCODE']

54313.0

In [66]:
east4_latitude = east_data.loc[7, 'LATITUDE']
east4_longitude = east_data.loc[7, 'LONGITUDE']

In [101]:
LIMIT = 100
radius = 10000

url4 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    east4_latitude, 
    east4_longitude, 
    radius, 
    LIMIT)
url4

'https://api.foursquare.com/v2/venues/explore?&client_id=4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3&client_secret=2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF&v=20180605&ll=44.5911,-88.1096&radius=10000&limit=100'

In [102]:
results = requests.get(url4).json()
results

{'meta': {'code': 200, 'requestId': '5c64f7361ed2192b5ba228c7'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4c75561208dcef3bb2301dcf-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d118951735',
         'name': 'Grocery Store',
         'pluralName': 'Grocery Stores',
         'primary': True,
         'shortName': 'Grocery Store'}],
       'id': '4c75561208dcef3bb2301dcf',
       'location': {'address': '2348 Lineville Rd',
        'cc': 'US',
        'city': 'Suamico',
        'country': 'United States',
        'distance': 2790,
        'formattedAddress': ['2348 Lineville Rd',
         'Suamico, WI 54313',
         'United States'],
        'labeledLatLngs': [{'label': 'display',
  

In [103]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Festival Foods,Grocery Store,44.592240,-88.074435
1,Olsen's Piggly Wiggly,Supermarket,44.589614,-88.079989
2,Anduzzis Sports Club Howard,Sports Bar,44.590448,-88.082819
3,The Bottle Room,Wine Bar,44.591669,-88.070024
4,Westside YMCA,Gym / Fitness Center,44.555886,-88.085911


In [104]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
American Restaurant,3,3,3
Arts & Crafts Store,1,1,1
Asian Restaurant,1,1,1
Bakery,2,2,2
Bank,1,1,1
Bar,6,6,6
Breakfast Spot,3,3,3
Burger Joint,1,1,1
Butcher,1,1,1


## Foursquare Exploration - West Side


In [106]:
# explore first zip code on the west side
west_data.loc[4, 'ZIPCODE']

54229.0

In [107]:
west1_latitude = west_data.loc[4, 'LATITUDE']
west1_longitude = west_data.loc[4, 'LONGITUDE']

In [108]:
LIMIT = 100
radius = 10000

url5 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    west1_latitude, 
    west1_longitude, 
    radius, 
    LIMIT)
url5

'https://api.foursquare.com/v2/venues/explore?&client_id=4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3&client_secret=2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF&v=20180605&ll=44.5486,-87.8353&radius=10000&limit=100'

In [110]:
results = requests.get(url5).json()
results

{'meta': {'code': 200, 'requestId': '5c64fce6f594df7dc9b10e99'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd344cc9854d13a1858fd4d-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/golfcourse_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e6941735',
         'name': 'Golf Course',
         'pluralName': 'Golf Courses',
         'primary': True,
         'shortName': 'Golf Course'}],
       'id': '4bd344cc9854d13a1858fd4d',
       'location': {'address': '4831 Church Rd',
        'cc': 'US',
        'city': 'New Franken',
        'country': 'United States',
        'distance': 705,
        'formattedAddress': ['4831 Church Rd',
         'New Franken, WI 54229',
         'United States'],
        'labeledLatLngs': [{'label': 'display',


In [111]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Royal Scot Golf Course,Golf Course,44.554290,-87.839206
1,The Settlement,American Restaurant,44.561960,-87.885978
2,Black Sheep Pub & Grill,Pub,44.539986,-87.898701
3,Wequiock Falls County Park,Park,44.567218,-87.892012
4,Weidner Center for the Performing Arts,Concert Hall,44.533061,-87.923029


In [112]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
American Restaurant,3,3,3
Bar,8,8,8
Baseball Field,1,1,1
Café,1,1,1
City,1,1,1
Coffee Shop,2,2,2
College Arts Building,1,1,1
Concert Hall,1,1,1
Convenience Store,1,1,1


In [113]:
#none fall under the italian restaurant category, move onto next zipcode

In [123]:
# explore second zip code on the west side
west_data.loc[5, 'ZIPCODE']

54301.0

In [114]:
west2_latitude = west_data.loc[5, 'LATITUDE']
west2_longitude = west_data.loc[5, 'LONGITUDE']

In [116]:
LIMIT = 100
radius = 10000

url6 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    west2_latitude, 
    west2_longitude, 
    radius, 
    LIMIT)
url6

'https://api.foursquare.com/v2/venues/explore?&client_id=4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3&client_secret=2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF&v=20180605&ll=44.4812,-88.0167&radius=10000&limit=100'

In [117]:
results = requests.get(url6).json()
results

{'meta': {'code': 200, 'requestId': '5c64fef46a60712d32090ac9'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b5af891f964a520dadc28e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1c9941735',
         'name': 'Ice Cream Shop',
         'pluralName': 'Ice Cream Shops',
         'primary': True,
         'shortName': 'Ice Cream'}],
       'id': '4b5af891f964a520dadc28e3',
       'location': {'address': '508 Greene Ave',
        'cc': 'US',
        'city': 'Green Bay',
        'country': 'United States',
        'distance': 1351,
        'formattedAddress': ['508 Greene Ave',
         'Green Bay, WI 54301',
         'United States'],
        'labeledLatLngs': [{'label': 'display',
          '

In [118]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Zesty's Frozen Custard,Ice Cream Shop,44.469532,-88.021385
1,Astor Park,Park,44.496511,-88.009396
2,Lorelei Inn,German Restaurant,44.493041,-88.016608
3,Happy Joe's Pizza & Ice Cream - Green Bay,Pizza Place,44.494676,-87.986419
4,Plae Bistro,American Restaurant,44.454529,-87.998724


In [119]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
American Restaurant,8,8,8
Asian Restaurant,1,1,1
Bagel Shop,1,1,1
Bakery,1,1,1
Bar,4,4,4
Baseball Field,1,1,1
Basketball Stadium,1,1,1
Beer Garden,1,1,1
Big Box Store,1,1,1


In [120]:
#none fall under the italian restaurant category, move onto next zipcode

In [121]:
# explore third zip code on the west side
west_data.loc[6, 'ZIPCODE']

54302.0

In [122]:
west3_latitude = west_data.loc[6, 'LATITUDE']
west3_longitude = west_data.loc[6, 'LONGITUDE']

In [124]:
LIMIT = 100
radius = 10000

url7 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    west3_latitude, 
    west3_longitude, 
    radius, 
    LIMIT)
url7

'https://api.foursquare.com/v2/venues/explore?&client_id=4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3&client_secret=2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF&v=20180605&ll=44.5088,-87.9776&radius=10000&limit=100'

In [125]:
results = requests.get(url7).json()
results

{'meta': {'code': 200, 'requestId': '5c6500c39fb6b72ae9af10f7'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-57f25240498ee3962874d011-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d118951735',
         'name': 'Grocery Store',
         'pluralName': 'Grocery Stores',
         'primary': True,
         'shortName': 'Grocery Store'}],
       'id': '57f25240498ee3962874d011',
       'location': {'address': '2430 University Avenue',
        'cc': 'US',
        'city': 'Green Bay',
        'country': 'United States',
        'distance': 1381,
        'formattedAddress': ['2430 University Avenue',
         'Green Bay, WI 54302',
         'United States'],
        'labeledLatLngs': [{'label':

In [126]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Festival Foods,Grocery Store,44.513174,-87.961315
1,Bay Beach Wildlife Sanctuary,Nature Preserve,44.524538,-87.964075
2,Happy Joe's Pizza & Ice Cream - Green Bay,Pizza Place,44.494676,-87.986419
3,Stillmank Brewing Co,Brewery,44.504538,-87.978232
4,Bay Area Humane Society,Pet Store,44.521338,-87.975972


In [127]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
American Restaurant,8,8,8
Bagel Shop,1,1,1
Bar,9,9,9
Baseball Field,1,1,1
Beer Garden,1,1,1
Big Box Store,1,1,1
Brewery,5,5,5
Burger Joint,3,3,3
Burrito Place,1,1,1


In [128]:
#there is 1 italian restaurant

In [129]:
# explore third zip code on the west side
west_data.loc[7, 'ZIPCODE']

54311.0

In [130]:
west4_latitude = west_data.loc[7, 'LATITUDE']
west4_longitude = west_data.loc[7, 'LONGITUDE']

In [131]:
LIMIT = 100
radius = 10000

url8 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    west4_latitude, 
    west4_longitude, 
    radius, 
    LIMIT)
url8

'https://api.foursquare.com/v2/venues/explore?&client_id=4BRJSOWJ3QQ5AX3XMYZO5KXRF4MAYRR35VNRIG1O320NWBU3&client_secret=2DSXIDEZVXIBG40O2VACXTBOTPWPJHUUEDNPLPR3W2VTPGWF&v=20180605&ll=44.4739,-87.8746&radius=10000&limit=100'

In [132]:
results = requests.get(url8).json()
results

{'meta': {'code': 200, 'requestId': '5c65035fdd57977f5b77ef2c'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b3fadd4f964a52006ac25e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d176941735',
         'name': 'Gym',
         'pluralName': 'Gyms',
         'primary': True,
         'shortName': 'Gym'}],
       'id': '4b3fadd4f964a52006ac25e3',
       'location': {'address': '1740 S Huron Rd',
        'cc': 'US',
        'city': 'Green Bay',
        'country': 'United States',
        'distance': 3558,
        'formattedAddress': ['1740 S Huron Rd',
         'Green Bay, WI 54311',
         'United States'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 44.469041558139025,
 

In [133]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,YMCA,Gym,44.469042,-87.918876
1,Rosati's Pizza Of Green Bay,Pizza Place,44.487037,-87.917610
2,Mackinaws Grill & Spirits,American Restaurant,44.486639,-87.936604
3,Luigi's Italian Bistro,Italian Restaurant,44.464701,-87.945691
4,Culver's,Fast Food Restaurant,44.486614,-87.935722


In [134]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
American Restaurant,8,8,8
Automotive Shop,1,1,1
Bar,8,8,8
Baseball Field,1,1,1
Beer Bar,1,1,1
Big Box Store,1,1,1
Bowling Alley,1,1,1
Breakfast Spot,1,1,1
Brewery,1,1,1


In [135]:
#there is 1 italian restaurant

## Top Venues of Each Zip Code of East Side

In [137]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [138]:
east_venues = getNearbyVenues(names=east_data['ZIPCODE'],
                                   latitudes=east_data['LATITUDE'],
                                   longitudes=east_data['LONGITUDE']
                                  )


54155.0
54303.0
54304.0
54313.0


In [143]:
print(east_venues.shape)
east_venues.head()

(92, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,54303.0,44.5432,-88.0608,Hughesnet Authorized Dealer,44.542618,-88.060023,Business Service
1,54303.0,44.5432,-88.0608,Deer Run Park,44.545891,-88.058739,Park
2,54303.0,44.5432,-88.0608,anduzzi's,44.538845,-88.062096,Bar
3,54304.0,44.4867,-88.0608,Seroogy's Chocolates,44.485840,-88.062602,Candy Store
4,54304.0,44.4867,-88.0608,Marcus Bay Park Cinema,44.484569,-88.062346,Movie Theater


In [141]:
east_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
54303.0,3,3,3,3,3,3
54304.0,88,88,88,88,88,88
54313.0,1,1,1,1,1,1


In [144]:
print('There are {} uniques categories.'.format(len(east_venues['Venue Category'].unique())))

There are 55 uniques categories.


In [146]:
# one hot encoding
east_onehot = pd.get_dummies(east_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
east_onehot['Neighborhood'] = east_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [east_onehot.columns[-1]] + list(east_onehot.columns[:-1])
east_onehot = east_onehot[fixed_columns]

east_onehot.head()

,Neighborhood,Accessories Store,Arts & Crafts Store,Auto Dealership,Bakery,Bar,Boutique,Business Service,Candy Store,Clothing Store,Coffee Shop,Cosmetics Shop,Department Store,Dessert Shop,Doctor's Office,Electronics Store,Farm,Fast Food Restaurant,Food Court,Furniture / Home Store,Gift Shop,Health Food Store,IT Services,Ice Cream Shop,Insurance Office,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lingerie Store,Mattress Store,Medical Center,Men's Store,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Park,Pharmacy,Piercing Parlor,Pizza Place,Sandwich Place,Shipping Store,Shoe Store,Smoke Shop,Snack Place,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Women's Store
0,54303.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,54303.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,54303.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,54304.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,54304.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [148]:
east_onehot.shape

(92, 56)

In [149]:
east_grouped = east_onehot.groupby('Neighborhood').mean().reset_index()
east_grouped

,Neighborhood,Accessories Store,Arts & Crafts Store,Auto Dealership,Bakery,Bar,Boutique,Business Service,Candy Store,Clothing Store,Coffee Shop,Cosmetics Shop,Department Store,Dessert Shop,Doctor's Office,Electronics Store,Farm,Fast Food Restaurant,Food Court,Furniture / Home Store,Gift Shop,Health Food Store,IT Services,Ice Cream Shop,Insurance Office,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lingerie Store,Mattress Store,Medical Center,Men's Store,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Park,Pharmacy,Piercing Parlor,Pizza Place,Sandwich Place,Shipping Store,Shoe Store,Smoke Shop,Snack Place,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Women's Store
0,54303.0,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,54304.0,0.034091,0.011364,0.011364,0.011364,0.000000,0.011364,0.000000,0.011364,0.079545,0.022727,0.022727,0.045455,0.011364,0.011364,0.011364,0.0,0.034091,0.011364,0.045455,0.011364,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.011364,0.022727,0.022727,0.011364,0.011364,0.045455,0.011364,0.034091,0.011364,0.000000,0.022727,0.011364,0.011364,0.022727,0.011364,0.056818,0.022727,0.022727,0.011364,0.011364,0.011364,0.011364,0.011364,0.022727,0.011364,0.011364,0.011364,0.011364,0.034091
2,54313.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [153]:
num_top_venues = 5

for hood in east_grouped['Neighborhood']:
    print(hood)
    temp = east_grouped[east_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

54303.0
               venue  freq
0                Bar  0.33
1   Business Service  0.33
2               Park  0.33
3  Accessories Store  0.00
4         Smoke Shop  0.00


54304.0
                    venue  freq
0          Clothing Store  0.08
1              Shoe Store  0.06
2             Men's Store  0.05
3  Furniture / Home Store  0.05
4        Department Store  0.05


54313.0
               venue  freq
0               Farm   1.0
1  Accessories Store   0.0
2     Lingerie Store   0.0
3     Medical Center   0.0
4        Men's Store   0.0




In [168]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [169]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = east_grouped['Neighborhood']

for ind in np.arange(east_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(east_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,54303.0,Bar,Business Service,Park,Women's Store,Electronics Store,Japanese Restaurant,Insurance Office,Ice Cream Shop,IT Services,Health Food Store
1,54304.0,Clothing Store,Shoe Store,Department Store,Furniture / Home Store,Men's Store,Women's Store,Fast Food Restaurant,Mobile Phone Shop,Accessories Store,Kids Store
2,54313.0,Farm,Women's Store,Doctor's Office,Japanese Restaurant,Insurance Office,Ice Cream Shop,IT Services,Health Food Store,Gift Shop,Furniture / Home Store


## Top Venues of Each Zip Code of West Side

In [154]:
west_venues = getNearbyVenues(names=west_data['ZIPCODE'],
                                   latitudes=west_data['LATITUDE'],
                                   longitudes=west_data['LONGITUDE']
                                  )

54229.0
54301.0
54302.0
54311.0


In [155]:
print(west_venues.shape)
west_venues.head()

(11, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,54301.0,44.4812,-88.0167,Webster Park,44.482051,-88.017957,Playground
1,54301.0,44.4812,-88.0167,Superior Discount Liquor,44.483700,-88.014855,Food & Drink Shop
2,54301.0,44.4812,-88.0167,Allouez/Bellin Health Farmers' Market,44.483372,-88.013685,Farmers Market
3,54302.0,44.5088,-87.9776,Stillmank Brewing Co,44.504538,-87.978232,Brewery
4,54302.0,44.5088,-87.9776,The Haystack,44.512667,-87.975059,Bar


In [157]:
west_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
54301.0,3,3,3,3,3,3
54302.0,8,8,8,8,8,8


In [159]:
print('There are {} uniques categories.'.format(len(west_venues['Venue Category'].unique())))

There are 11 uniques categories.


In [160]:
# one hot encoding
west_onehot = pd.get_dummies(west_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
west_onehot['Neighborhood'] = west_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [west_onehot.columns[-1]] + list(west_onehot.columns[:-1])
west_onehot = west_onehot[fixed_columns]

west_onehot.head()

,Neighborhood,Bakery,Bar,Brewery,Department Store,Farmers Market,Food & Drink Shop,Mexican Restaurant,Pawn Shop,Pizza Place,Playground,Sports Bar
0,54301.0,0,0,0,0,0,0,0,0,0,1,0
1,54301.0,0,0,0,0,0,1,0,0,0,0,0
2,54301.0,0,0,0,0,1,0,0,0,0,0,0
3,54302.0,0,0,1,0,0,0,0,0,0,0,0
4,54302.0,0,1,0,0,0,0,0,0,0,0,0


In [162]:
west_onehot.shape

(11, 12)

In [163]:
west_grouped = west_onehot.groupby('Neighborhood').mean().reset_index()
west_grouped

,Neighborhood,Bakery,Bar,Brewery,Department Store,Farmers Market,Food & Drink Shop,Mexican Restaurant,Pawn Shop,Pizza Place,Playground,Sports Bar
0,54301.0,0.000,0.000,0.000,0.000,0.333333,0.333333,0.000,0.000,0.000,0.333333,0.000
1,54302.0,0.125,0.125,0.125,0.125,0.000000,0.000000,0.125,0.125,0.125,0.000000,0.125


In [164]:
num_top_venues = 5

for hood in west_grouped['Neighborhood']:
    print(hood)
    temp = west_grouped[west_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

54301.0
               venue  freq
0     Farmers Market  0.33
1  Food & Drink Shop  0.33
2         Playground  0.33
3             Bakery  0.00
4                Bar  0.00


54302.0
                venue  freq
0              Bakery  0.12
1                 Bar  0.12
2             Brewery  0.12
3    Department Store  0.12
4  Mexican Restaurant  0.12




In [170]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = west_grouped['Neighborhood']

for ind in np.arange(west_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(west_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,54301.0,Playground,Food & Drink Shop,Farmers Market,Sports Bar,Pizza Place,Pawn Shop,Mexican Restaurant,Department Store,Brewery,Bar
1,54302.0,Sports Bar,Pizza Place,Pawn Shop,Mexican Restaurant,Department Store,Brewery,Bar,Bakery,Playground,Food & Drink Shop
